In [1]:
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import pandas as pd
import base64
import csv
import io
import os
from CRUD_module import AnimalShelter

# Parse raw CSV-like records into dictionaries
def parse_record(record):
    if isinstance(record, dict):
        # Extract the key (which contains the CSV headers) and the value (CSV row)
        key = list(record.keys())[0]  # Get the CSV header row (incorrectly as key)
        value = record[key]           # Get the CSV row as value
        
        # Parse the CSV header and row
        csv_reader = csv.DictReader(io.StringIO(f"{key}\n{value}"))
        return list(csv_reader)[0]  # Convert to dictionary
    
    return record

# Database Configuration
try:
    db = AnimalShelter()
    raw_data = db.read({})  # Fetch all records

    # Debugging: Print raw records to verify structure
    print("Raw Records from MongoDB:")
    for record in raw_data[:5]:  # Print only the first 5 records
        print(record)

    # Parse records and create DataFrame
    parsed_data = [parse_record(record) for record in raw_data]
    df = pd.DataFrame(parsed_data)

    # Debugging: Print the parsed DataFrame
    print("Parsed DataFrame Columns:", df.columns)
    print(df.head())

    # Reindex dynamically to match data structure
    df = df.reindex(columns=df.columns.tolist(), fill_value='N/A')

    # Strip whitespace from column names
    df.columns = df.columns.str.strip()
    df = df.astype(str)
    print("Final DataFrame Columns:", df.columns)
    print("First Row:", df.iloc[0])

except Exception as e:
    print(f"Error fetching or cleaning data: {e}")
    df = pd.DataFrame()

# Initialize Dash App
app = JupyterDash(__name__)

# Encode the Grazioso Salvare logo
image_filename = 'Grazioso_Salvare_Logo.png'
try:
    encoded_image = base64.b64encode(open(image_filename, 'rb').read())
except FileNotFoundError:
    encoded_image = None

# Dynamic Dash Table Columns
if not df.empty:
    table_columns = [{"name": i, "id": i} for i in df.columns]
else:
    table_columns = []

# App Layout
app.layout = html.Div([
    html.Center(html.H1("Grazioso Salvare Dashboard")),
    html.Center(
        html.Img(src=f'data:image/png;base64,{encoded_image.decode()}', style={'width': '150px'}) if encoded_image else "Logo Missing"
    ),
    html.Center(html.H5("Created by: Caleb Green")),
    html.Hr(),
    html.Label("Select Rescue Type:"),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain/Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
            {'label': 'Disaster/Individual Tracking', 'value': 'Disaster or Individual Tracking'},
            {'label': 'Reset', 'value': 'Reset'}
        ],
        value='Reset',
        inline=True
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=table_columns,
        data=df.to_dict('records'),
        page_size=10,
        style_table={'height': '400px', 'overflowY': 'auto'},
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold',
            'textAlign': 'center',
            'border': '1px solid black'
        },
        style_data={
            'whiteSpace': 'normal',
            'height': 'auto',
            'border': '1px solid black',
            'textAlign': 'left',
            'padding': '10px'
        },
        style_cell={
            'minWidth': '120px', 'maxWidth': '300px', 'width': '200px',
            'overflow': 'hidden',
            'textOverflow': 'ellipsis'
        }
    ),
    html.Br(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

@app.callback(
    Output('datatable-id', 'data'),
    Input('filter-type', 'value')
)
def update_table(filter_type):
    if df.empty:
        return []
    if filter_type == 'Reset':
        return df.to_dict('records')
    elif filter_type == 'Water Rescue':
        filtered_df = df[df['outcome_type'] == 'Water Rescue']
    elif filter_type == 'Mountain or Wilderness Rescue':
        filtered_df = df[df['outcome_type'] == 'Mountain or Wilderness Rescue']
    elif filter_type == 'Disaster or Individual Tracking':
        filtered_df = df[df['outcome_type'] == 'Disaster or Individual Tracking']
    else:
        filtered_df = df

    return filtered_df.to_dict('records')

if __name__ == "__main__":
    app.run_server(debug=True)


Raw Keys in MongoDB Document: ['"rec_num","age_upon_outcome","animal_id","animal_type","breed","color","date_of_birth","datetime","monthyear","name","outcome_subtype","outcome_type","sex_upon_outcome","location_lat","location_long","age_upon_outcome_in_weeks"']
Raw Records from MongoDB:
{'rec_num,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome,location_lat,location_long,age_upon_outcome_in_weeks': '"1","3 years","A746874","Cat","Domestic Shorthair Mix","Black/White",2014-04-10,2017-04-11 09:00:00,"2017-04-11T09:00:00","","SCRP","Transfer","Neutered Male",30.5066578739455,-97.3408780722188,156.767857142857'}
{'rec_num,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome,location_lat,location_long,age_upon_outcome_in_weeks': '"2","1 year","A725717","Cat","Domestic Shorthair Mix","Silver Tabby",2015-05-02,2016-05-06 10:49: